In [1]:
import cv2
import numpy as np
from collections import deque

def extract_and_compress_adaptive(video_path, output_video_path, interval_sec=5, codec='XVID', fps_window=30):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    nominal_fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(f"Nominal FPS from video: {nominal_fps}")
    
    # Sanity check for FPS
    if nominal_fps <= 0 or nominal_fps > 1000:  # Unreasonable FPS values
        nominal_fps = 30  # Default to standard fps
        print(f"Warning: Invalid FPS detected, defaulting to {nominal_fps}")
    
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*codec)
    output_fps = 24
    out = cv2.VideoWriter(output_video_path, fourcc, output_fps, (width, height))

    # Variables for tracking real time
    frame_count = 0
    output_frame_count = 0
    frame_times = deque(maxlen=fps_window)
    last_frame_time = None
    processing_start_time = cv2.getTickCount()
    
    def estimate_current_time():
        """Estimate current video time using multiple methods"""
        msec_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
        frame_based_time = frame_count / nominal_fps
        
        # If msec_time seems reasonable, use it
        if msec_time > 0 and msec_time < frame_based_time * 2:
            return msec_time
        return frame_based_time
    
    def calculate_local_fps(times):
        if len(times) < 2:
            return nominal_fps
        time_diff = times[-1] - times[0]
        if time_diff <= 0:
            return nominal_fps
        return (len(times) - 1) / time_diff

    next_sample_time = 0.0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        frame_count += 1
        current_time = estimate_current_time()
        frame_times.append(current_time)
        
        # Calculate local FPS
        local_fps = calculate_local_fps(list(frame_times))
        
        # Adjust sampling interval based on local FPS
        target_fps = 30.0
        adjustment_factor = local_fps / target_fps
        adaptive_interval = interval_sec * adjustment_factor
        
        # Sample frame if needed
        if current_time >= next_sample_time:
            out.write(frame)
            output_frame_count += 1
            next_sample_time = current_time + adaptive_interval
            
            # Print progress every 100 frames
            if output_frame_count % 100 == 0:
                print(f"\rProcessed {frame_count}/{total_frames} frames. "
                      f"Time: {current_time:.2f}s, Local FPS: {local_fps:.1f}", end="")

    # Calculate final statistics
    processing_time = (cv2.getTickCount() - processing_start_time) / cv2.getTickFrequency()
    estimated_duration = frame_count / nominal_fps
    output_duration = output_frame_count / output_fps
    
    print(f"\n\nProcessing Statistics:")
    print(f"Original frames: {frame_count}")
    print(f"Output frames: {output_frame_count}")
    print(f"Nominal input FPS: {nominal_fps:.2f}")
    print(f"Estimated original duration: {estimated_duration:.2f} seconds")
    print(f"Output duration: {output_duration:.2f} seconds")
    print(f"Processing time: {processing_time:.2f} seconds")
    print(f"Average processing FPS: {frame_count/processing_time:.2f}")

    # Cleanup
    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Usage example
video_path = '../data/S2-dissect12-14-video-1-2-100xmag-Video30.mov'
output_video_path = '../data/S2_Video30.mp4'
extract_and_compress_adaptive(video_path, output_video_path, interval_sec=2, codec='mp4v')

Nominal FPS from video: 1.2162352628547088
Processed 103028/103398 frames. Time: 84864.10s, Local FPS: 0.2

Processing Statistics:
Original frames: 103398
Output frames: 18651
Nominal input FPS: 1.22
Estimated original duration: 85014.80 seconds
Output duration: 777.12 seconds
Processing time: 163.25 seconds
Average processing FPS: 633.37
